<a href="https://colab.research.google.com/github/Devauntae/-My-Cornell-Tech-Portfolio/blob/main/TradingBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lumibot==2.9.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.9/513.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import trader
from datetime import datetime

In [ ]:
API_KEY = "PKB9JQZLXJB7D3N4I2D9"
API_SECRET = "kbN1CQwZssOQOnkEq55UshrHvgDzMxKff9ZvQCwI"
BASE_URL = "https://paper-api.alpaca.markets/v2"

ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER": True
}

In [ ]:
class MLTrader(Strategy):
  def initialize(self, symbol:str="SPY"):
    self.symbol = symbol
    self.sleeptime = "24H"
    self.last_trade = None

  def on_trading_iteration(self):
    if self.last_trade == None:
      order = self.create_order(
          self.symbol,
          10,
          "buy",
          type = "market"
      )
      self.submit_order(order)
      self.last_trade = "buy"


start_date = datetime(2025,2,1)
end_date = datetime(2025,3,1)

broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name ='mlstrat', broker=broker, parameters={"symbol":"SPY"})
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY"}
)


ERROR:root:Error getting the risk free rate: Too Many Requests. Rate limited. Try after a while.


Starting backtest for MLTrader...


ERROR:root:Error getting the risk free rate: Too Many Requests. Rate limited. Try after a while.
INFO:backtest_stats:Starting backtest...


Progress ||   8.56%  [Elapsed: 0:00:02 ETA: 0:00:24] 


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.11/dist-packages/lumibot/tools/decorators.py", line 28, in call_function_get_frame
    sys.settrace(snatch_locals)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.11/dist-packages/lumibot/tools/decorators.py", line 32, in call_function_get_frame
    sys.settrace(trace)



2025-02-19 04:09:12,952: root: ERROR: YAHOO did not return data for symbol SPY. Make sure this symbol is valid.
2025-02-19 04:09:12,954: root: ERROR: 'NoneType' object has no attribute 'df'
2025-02-19 04:09:12,956: root: ERROR: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/lumibot/strategies/strategy_executor.py", line 829, in run
    self._run_trading_session()
  File "/usr/local/lib/python3.11/dist-packages/lumibot/strategies/strategy_executor.py", line 783, in _run_trading_session
    self.broker.process_pending_orders(strategy=self.strategy)
  File "/usr/local/lib/python3.11/dist-packages/lumibot/backtesting/backtesting_broker.py", line 513, in process_pending_orders
    dt = ohlc.df.index[-1]
         ^^^^^^^
AttributeError: 'NoneType' object has no attribute 'df'

2025-02-19 04:09:13,627: root: ERROR: 'NoneType' object has no attribute 'loc'
2025-02-19 04:09:13,631: root: ERROR: Traceback (most recent call last):
  File "/usr/local/lib/python3

Exception in thread MLTrader:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/lumibot/strategies/strategy_executor.py", line 829, in run
    self._run_trading_session()
  File "/usr/local/lib/python3.11/dist-packages/lumibot/strategies/strategy_executor.py", line 783, in _run_trading_session
    self.broker.process_pending_orders(strategy=self.strategy)
  File "/usr/local/lib/python3.11/dist-packages/lumibot/backtesting/backtesting_broker.py", line 513, in process_pending_orders
    dt = ohlc.df.index[-1]
         ^^^^^^^
AttributeError: 'NoneType' object has no attribute 'df'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/dist-packages/lumibot/strategies/strategy_executor.py", line 842, in run
    raise RuntimeError("Exception encountered, stopping BackTest.") from e
Runt

2025-02-19 04:09:13,638: root: INFO: Backtesting finished
2025-02-19 04:09:13,643: root: WARNING: Cannot plot returns because the benchmark returns are missing

Creating indicators plot...

Creating tearsheet...
2025-02-19 04:09:13,971: root: ERROR: No data to create tearsheet, skipping
2025-02-19 04:09:13,972: backtest_stats: INFO: Backtest took 0:00:03.138974 for a speed of 0.000


{}